# From Collections Data to Linked Art - Visualisation - John Ruskin

This notebook is concerned with a visualisation of artworks represented in Linked Art.


## Input Data

### Cleveland Museum of Art 
http://www.clevelandart.org/
- Download CSV file from GitHub https://github.com/ClevelandMuseumArt/openaccess
- View file with OpenRefine https://openrefine.org/
- Create a text facet for the Creator field to identify artworks by John Ruskin
- Export this subset of records (1 record found)
- [data file](./data/cma/input/ruskin.csv) 


### National Gallery of Art

- [data file](./data/nga/input/nga_ruskin.csv)

In [123]:
try:
    import ipywidgets as widgets
except:
    !pip install ipywidgets
    import ipywidgets as widgets

from ipywidgets import Layout
from ipywidgets import FileUpload

try:
    import IPython
except:
    !pip install IPython
    import IPython   
    
from IPython.display import display
from IPython.core.display import HTML
from IPython.display import IFrame

   
try:
    import xmltodict
except:
    !pip install xmltodict
    import xmltodict

try:
    import json
except:
    !pip install json
    import json 
    
    

try:
    import requests
except:
    !pip install requests
    import requests

import csv

In [124]:
import csv

def getObjDesc(obj, functionCall, mapp):
    
    desc = {}
    
    id = obj[mapp.get("id")]
    irn = obj[mapp.get("irn")]
    department  = obj[mapp.get("department")]
    title = obj[mapp.get("title")]
    datecreated = obj[mapp.get("datecreated")]
    medium = obj[mapp.get("medium")]
    classification = obj[mapp.get("classification")]
    attribution = obj[mapp.get("attribution")]
    credit_line = obj[mapp.get("credit_line")]
    titAccessionNo = obj[mapp.get("titAccessionNo")]
    if mapp.get("provenance") in list(obj.keys()):
        provenance = obj[mapp.get("provenance")]
    else:
        provenance = ""
    earliestdate = obj[mapp.get("earliestdate")] 
    latestdate = obj[mapp.get("latestdate")]
    
    if functionCall == "member":
        desc = objMember(obj,baseURI,PhyCollectionArea)
    if functionCall == "custody":
        desc = objCustody(TitObjectStatus)
    if functionCall == "owner":
        desc = objOwner(obj,baseURI,irn,TitAccessionDate,TitObjectStatus)
    if functionCall == "production":
        desc = objProd(obj, baseURI,irn, earliestdate,latestdate,datecreated)
    if functionCall == "core":
        desc = objCore(obj,baseURI,irn,title) 
    if functionCall == "id":
        desc = objId(obj, baseURI, irn, titAccessionNo)
    if functionCall == "names":
        desc = objNames(obj, baseURI, irn, title)
    if functionCall == "class":
        if "titleObjectType" in vars():
            desc = objClass(obj, baseURI, irn, titleObjectType)
    if functionCall == "home":
            desc = objHome(obj,baseURI,irn)
    if functionCall == "location":
            desc = objLocation(obj,baseURI)
    if functionCall == "ling":
            desc = objLing(obj,baseURI,irn, credit_line, provenance)
    return desc



def getAllObjects(file, mapp):
    
    
    #remove BOM
    s = open(file, mode='r', encoding='utf-8-sig').read()
    open(file, mode='w', encoding='utf-8').write(s)
    allObjects = csv.DictReader(open(file, mode='r',encoding='utf-8'))
    all = []
    for obj in allObjects:
        id = obj[mapp.get("id")]
        all.append(obj)  
    return all


def objCore(obj, baseURI, irn, title):
    core = {}

        # minimum Linked Art properties
    core["@context"] = "https://linked.art/ns/v1/linked-art.json"
    core["id"] = baseURI + irn
    core["type"] = "HumanMadeObject"
    if "title" in vars():
        core["_label"] = title 
    
    return core


def objId(obj, baseURI, irn, titAccessionNo):
    artwork = {}       
    artwork["identified_by"] = []          
    if titAccessionNo != "":
        artwork["identified_by"].append({
        "id": baseURI + "object/" + irn + "/object-number",
        "type": "Identifier",
        "_label": "Object Number for the Object",
        "content": titAccessionNo,
        "classified_as": [{
            "id": "http://vocab.getty.edu/aat/300312355",
            "type": "Type",
            "_label": "accession numbers"
                        }]
                })
        
    return artwork

def objNames(obj, baseURI, irn, title):

    artwork = {}
    artwork["identified_by"] = []
    
    artwork["identified_by"].append({
        "id": baseURI + "object/" + irn + "/title",
        "type": "Name",
        "_label": "Primary Title for the Object",
        "content": title ,
        "classified_as": [{
        "id": "http://vocab.getty.edu/aat/300404670",
        "type": "Type",
        "_label": "preferred terms"
                        }]
                })
    try:   
        if obj["table"]["@name"] == "AltTitles":
            x = 0
            for tuple in obj["table"]["tuple"]:
                x +=1
                for atom in tuple:
                    content = ""
                    if atom["@name"] == "TitAlternateTitles":
                        content = atom["#text"]
                        artwork["identified_by"].append({
                            "id": baseURI + "object/" + irn + "/alt-title-" + x,
                            "type": "Name",
                            "_label": "Alternate Title for the Object",
                            "content": content,
                            "classified_as": [{
                                "id": "http://vocab.getty.edu/aat/300417227",
                                "type": "Type",
                                "_label": "alternate titles"}]   
                        })
    except:
        pass

    return artwork


def objLing(obj, baseURI,irn, SumCreditLine, provenance):
    artwork = {}
    artwork["referred_to_by"] = []
    if SumCreditLine != "":
        artwork["referred_to_by"].append(
                {
                "id": baseURI + "object/" + irn + "/credit-line",
                "type": "LinguisticObject",
                "_label": "Credit Line for the Object",
                "content": SumCreditLine,
                "classified_as": [
                        {
                        "id": "http://vocab.getty.edu/aat/300026687",
                        "type": "Type",
                        "_label": "acknowledgments"
                        },
                        {
                        "id": "http://vocab.getty.edu/aat/300418049",
                        "type": "Type",
                        "_label": "brief texts"
                        }]
                })
              
    if provenance != "":
        artwork["referred_to_by"].append({
                "id": baseURI + "object/" + irn + "/provenance-statement",
                    "type": "LinguisticObject",
                    "_label": "Provenance Statement about the Object",
                    "content": provenance,
                    "classified_as": [
                        {
                            "id": "http://vocab.getty.edu/aat/300055863",
                            "type": "Type",
                            "_label": "provenance (history of ownership)"
                        },
                        {
                            "id": "http://vocab.getty.edu/aat/300418049",
                            "type": "Type",
                            "_label": "brief texts"
                        }
                    ]
            })
        
            
    return artwork

def objProd(obj, baseURI,irn, earliestdate,latestdate,datecreated):
    
    if "attribution" in list(obj.keys()):
        attr = obj["attribution"]
    else:
        attr = ""
    
    artwork = {}
    
    #produced_by property
    artwork["produced_by"] = []
    artwork["produced_by"].append({
                 "id": baseURI + "object/" + irn + "/production",
                "type": "Production",
                "_label": "Production of the Object"})
    
    #carried_out_by property
    carried_out_by = []
    carried_out_by.append(
                                {
                                "id":  baseURI + "actor/" + irn,
                                "type": "Actor",
                                "_label": attr
                                }
                            )        
                                  
    if len(carried_out_by) > 0:
        artwork["produced_by"][0].update(carried_out_by = carried_out_by)
                    

                
    # timespan property
    timespan = False
    
    
    for date in (earliestdate,latestdate,datecreated):
        if date != "":
            timespan = True
            
    if timespan == True:
        #label
        label = "date unknown"
        if datecreated != "":
            label = datecreated
        elif (earliestdate != "") or (latestdate != ""):
            label = earliestdate + " - " + latestdate
        
        timespanObj = {
               "id": baseURI + irn + "/production/timespan",
                "type": "TimeSpan",
                "_label": label,
            }
        
        if earliestdate != "":
            timespanObj["begin_of_the_begin"] = earliestdate
    
        if latestdate != "":
            timespanObj["end_of_the_end"] = latestdate
        
        artwork["produced_by"][0].update(timespan = timespanObj)
        
    return artwork

def objCustody(TitObjectStatus):
    
    currentowner = False
    artwork = {}
    
    checkObjStatus = ('Accessioned','Partial Accession')
    for status in checkObjStatus:
        if status == TitObjectStatus:
            currentowner = True
    if 'IMA-Owned' in TitObjectStatus:
            currentowner = True
            
    if currentowner == False:  
        artwork["current_keeper"] =  {
                "id": "http://vocab.getty.edu/ulan/500300517",
                "type": "Group",
                "_label": "PMA",
                "classified_as": [
                    {
                        "id": "http://vocab.getty.edu/aat/300312281",
                        "type": "Type",
                        "_label": "museums (institutions)"
                    }]}
    
    return artwork


In [125]:
#  baseURI for JSON-LD document
baseURI = "https://www.nga.gov/collection/"

file = './data/nga/input/nga_ruskin.csv'

all_linkedart = {}

mapp = {
    "id" : "objectid",
    "irn" : "accessionnum",
    "department" : "departmentabbr",
    "title" : "title",
    "datecreated" : "displaydate",
    "medium" : "medium",
    "classification" : "classification",
    "attribution" : "attribution",
    "credit_line" : "creditline",
    "titAccessionNo" : "accessionnum",
    "provenance" : "none",
    "earliestdate" : "beginyear",
    "latestdate" : "endyear"
      
}
 
    
allObjects = getAllObjects(file, mapp)


for obj in allObjects:
    idf = mapp.get("id")
    irn = obj[idf]
    core = getObjDesc(obj, "core", mapp)
    all_linkedart[irn] = core 
    
    #identifiers
    id = getObjDesc(obj, "id", mapp)
    all_linkedart[irn].update(id)
    
    #names
    desc = getObjDesc(obj, "names", mapp)
    all_linkedart[irn].update(desc) 
    
    #classification
    desc = getObjDesc(obj, "class",mapp)
    if "classified_as" in list(desc.keys()):
        all_linkedart[irn].update(desc)
      
    # homepage
    #desc = getObjDesc(obj, "home")
    #all_linkedart[irn].update(desc) 
    
    # location
   # desc = getObjDesc(obj, "location")
   # all_linkedart[irn].update(desc) 
    
    # linguistic objects
    desc = getObjDesc(obj, "ling",mapp)
    all_linkedart[irn].update(desc) 
    
    # production
    desc = getObjDesc(obj, "production",mapp) 
    all_linkedart[irn].update(desc) 
    
    # ownership
   # desc = getObjDesc(obj, "owner")
   # all_linkedart[irn].update(desc)  
    
    #custody
   # desc = getObjDesc(obj, "custody")
   # if "current_keeper" in list(desc.keys()):
   #     all_linkedart[irn].update(desc)  
    
    # member 
  #  desc = getObjDesc(obj, "member")
  #  all_linkedart[irn].update(desc) 
    
for id in all_linkedart:
    text_file = open("./data/ruskin/output/" + id + ".json", "w")
    n = text_file.write(json.dumps(all_linkedart[id], indent=2))
    text_file.close()

In [126]:
import os
from IPython.core.display import display, HTML


def fn():       # 1.Get file names from directory
    file_list=os.listdir(r"./data/ruskin/output/")
   
    for file in file_list:
        display(HTML("<a target='_new' href='./data/ruskin/output/" + file +"'>" + file + "</a>"))
fn()

## CMA data

In [127]:
#  baseURI for JSON-LD document
baseURI = "https://clevelandart.org/art/"

file = './data/cma/input/ruskin.csv'


all_linkedart = {}

mapp = {
    "id" : "id",
    "irn" : "accession_number",
    "department" : "department",
    "title" : "title",
    "datecreated" : "creation_date",
    "medium" : "support_materials",
    "classification" : "type",
    "attribution" : "creators",
    "credit_line" : "creators",
    "titAccessionNo" : "accession_number",
    "provenance" : "provenance",
    "earliestdate" : "creation_date_earliest",
    "latestdate" : "creation_date_latest"
      
}


allObjects = getAllObjects(file, mapp)


for obj in allObjects:
    irn = obj[mapp.get("id")]
    core = getObjDesc(obj, "core", mapp)
    all_linkedart[irn] = core 
    
    #identifiers
    id = getObjDesc(obj, "id", mapp)
    all_linkedart[irn].update(id)
    
    #names
    desc = getObjDesc(obj, "names", mapp)
    all_linkedart[irn].update(desc) 
    
    #classification
    desc = getObjDesc(obj, "class", mapp)
    if "classified_as" in list(desc.keys()):
        all_linkedart[irn].update(desc)
      
    # homepage
    #desc = getObjDesc(obj, "home")
    #all_linkedart[irn].update(desc) 
    
    # location
   # desc = getObjDesc(obj, "location")
   # all_linkedart[irn].update(desc) 
    
    # linguistic objects
    desc = getObjDesc(obj, "ling", mapp)
    all_linkedart[irn].update(desc) 
    
    # production
    desc = getObjDesc(obj, "production", mapp) 
    all_linkedart[irn].update(desc) 
    
    # ownership
   # desc = getObjDesc(obj, "owner")
   # all_linkedart[irn].update(desc)  
    
    #custody
   # desc = getObjDesc(obj, "custody")
   # if "current_keeper" in list(desc.keys()):
   #     all_linkedart[irn].update(desc)  
    
    # member 
  #  desc = getObjDesc(obj, "member")
  #  all_linkedart[irn].update(desc) 
    
for id in all_linkedart:
    text_file = open("./data/ruskin/output/" + id + ".json", "w")
    n = text_file.write(json.dumps(all_linkedart[id], indent=2))
    text_file.close()

In [128]:
import os
from IPython.core.display import display, HTML


def fn():       # 1.Get file names from directory
    file_list=os.listdir(r"./data/ruskin/output/")
   
    for file in file_list:
        display(HTML("<a target='_new' href='./data/ruskin/output/" + file +"'>" + file + "</a>"))
fn()

## Convert JSON file to CSV

In [129]:
fpath = "./data/ruskin/output/"
file_list=os.listdir(r"./data/ruskin/output/")
  
json_all = {}
for file in file_list:
    # read file and append to 
    with open(fpath + file) as json_file:
        json_text = json.load(json_file)
        json_all.update({file : json_text})
        


for file in json_all:
    #print(json.dumps(json_all[file], indent=2))
    artwork = json_all[file]
    id = artwork["id"]
    
    
    csv = {}
    csv["id"] = artwork["id"]
    csv["title"] = artwork["identified_by"][0]["content"]
    csv["year_begin"] = artwork["produced_by"][0]["timespan"]["begin_of_the_begin"]
    csv["year_end"] = artwork["produced_by"][0]["timespan"]["end_of_the_end"]
    
    print(csv)   

{'id': 'https://clevelandart.org/art/1989.14', 'title': 'Budding Sycamore', 'year_begin': '1871', 'year_end': '1881'}
{'id': 'https://www.nga.gov/collection/1988.20.38', 'title': 'Tree Study', 'year_begin': '1845', 'year_end': '1855'}
{'id': 'https://www.nga.gov/collection/1987.73.2', 'title': 'Tower of the Cathedral at Sens', 'year_begin': '1845', 'year_end': '1845'}
{'id': 'https://www.nga.gov/collection/1995.52.158', 'title': 'Ornamental Study with Acanthus Motif for "The Stones of Venice"', 'year_begin': '1849', 'year_end': '1849'}
{'id': 'https://www.nga.gov/collection/1991.88.1', 'title': 'The Garden of San Miniato near Florence', 'year_begin': '1845', 'year_end': '1845'}
